# Data Exploration

In [ ]:
import math

import matplotlib.pyplot as plt
import pandas as pd


## Load data from csv File and output key insides

In [ ]:
# Load and inspect data
df = pd.read_csv("../data/raw/angle_estimation_dataset.csv")

print(f"Dataset shape: {df.shape}")
print("\n First 5 rows:")
display(df.head())

print("\n Column dtypes:")
print(df.dtypes)

print("\n Basic statistics:")
display(df.describe())

In [ ]:
# Check for missing values.
print("Missing Values Check:")
print("="*40)
missing = df.isnull().sum()

display(missing)

In [ ]:
# Check for duplicates and delete if any.
print(f"\nDuplicates: {df.duplicated().sum()}")
if df.duplicated().sum() > 0:
    print(f"Found {df.duplicated().sum()} duplicate rows")
    df = df.drop_duplicates()
    print("Duplicates removed")

In [ ]:
# Check if statistics change over time
print("Time-based Analysis (Check for Stationarity):")

# Split data into quarters
n_quarters = 6
quarter_size = len(df) // n_quarters

fig, ax = plt.subplots(2, 2, figsize=(12, 10))
ax = ax.flatten()

key_vars = ['i_alpha', 'i_beta', 'sin_theta_e', 'cos_theta_e']

for i, var in enumerate(key_vars):
    for q in range(n_quarters):
        start = q * quarter_size
        end = (q + 1) * quarter_size if q < n_quarters - 1 else len(df)
        ax[i].hist(df[var].iloc[start:end], bins=30, alpha=0.5,
                  label=f'Q{q+1}', density=True)
    ax[i].set_xlabel(var)
    ax[i].set_ylabel('Density')
    ax[i].set_title(f'{var} Distribution by Time sixth')
    ax[i].legend(fontsize=9)
    ax[i].grid(True, alpha=0.3)

plt.suptitle('Time Stationarity Check', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("\n If distributions look similar across sixth → Data is stationary")
print(" If distributions change → Non-stationary (time-dependent patterns)")

## Ouput Typical FOC data (Timeseries Data)
Here are the most common foc outputs. Here we can see how good the control algorthm is working. When reference and measured values are nearly the same, the control algorithm is working good.

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(14, 12))

# Plot 1: d-axis Current^
ax[0, 0].plot(df['time'], df['i_d_ref'], 'b--', alpha=0.8, label='id-ref')
ax[0, 0].plot(df['time'], df['i_d'], 'b-', alpha=0.5, label='id-measured', linewidth=1)
ax[0, 0].set_ylabel('Current [A]')
ax[0, 0].set_title('d-axis Current')
ax[0, 0].legend(fontsize=9)

# Plot 2: q-axis Current
ax[0, 1].plot(df['time'], df['i_q_ref'], 'r--', alpha=0.8, label='iq-ref')
ax[0, 1].plot(df['time'], df['i_q'], 'r-', alpha=0.5, label='iq-measured', linewidth=1)
ax[0, 1].set_ylabel('Current [A]')
ax[0, 1].set_title('q-axis Current')
ax[0, 1].legend(fontsize=9)

# Plot 3: Speed
ax[1, 0].plot(df['time'], df['omega_ref'], 'k--', alpha=0.5, label='Reference')
ax[1, 0].plot(df['time'], df['omega_e'], 'g-', alpha=0.5, label='Measured', linewidth=1)
ax[1, 0].set_ylabel('Speed [rad/s]')
ax[1, 0].set_title('Electrical Speed')
ax[1, 0].legend(fontsize=9)

# Plot 4: Torque
ax[1, 1].plot(df['time'], df['torque_e'], 'purple', alpha=0.8, label='Electromagnetic')
ax[1, 1].plot(df['time'], df['torque_load'], 'orange', alpha=0.8, label='Load')
ax[1, 1].set_ylabel('Torque [N·m]')
ax[1, 1].set_title('Torque')
ax[1, 1].legend(fontsize=9)

# Plot 5: Voltages
ax[2, 0].plot(df['time'], df['v_d'], 'b-', alpha=0.8, label='v_d')
ax[2, 0].plot(df['time'], df['v_q'], 'r-', alpha=0.8, label='v_q')
ax[2, 0].set_xlabel('Time [s]')
ax[2, 0].set_ylabel('Voltage [V]')
ax[2, 0].set_title('d-q Voltages')
ax[2, 0].legend(fontsize=9)


plt.suptitle('PMSM Signal Overview', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Distribution Data
In the following plots there are plots of the key foc data points. With that plots it can be seen in which operating points the motor is most of the time.

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(15, 10))

# i_d distribution
ax[0, 0].hist(df['i_d'], bins=50, alpha=0.7, label='meas')
ax[0, 0].set_xlabel('Current [A]')
ax[0, 0].set_ylabel('Count')
ax[0, 0].set_title('i_d Distribution')
ax[0, 0].legend()

# i_q distribution
ax[0, 1].hist(df['i_q'], bins=50, alpha=0.7, label='meas', color='red')
ax[0, 1].set_xlabel('Current [A]')
ax[0, 1].set_title('i_q Distribution')
ax[0, 1].legend()

# Speed distribution
ax[0, 2].hist(df['omega_e'], bins=50, alpha=0.7, label='True', color='green')
ax[0, 2].set_xlabel('Speed [rad/s]')
ax[0, 2].set_title('ω_e Distribution')
ax[0, 2].legend()

# Torque distribution
ax[1, 0].hist(df['torque_e'], bins=50, alpha=0.7, color='purple')
ax[1, 0].set_xlabel('Torque [N·m]')
ax[1, 0].set_ylabel('Count')
ax[1, 0].set_title('Electromagnetic Torque')

# Load torque distribution
ax[1, 1].hist(df['torque_load'], bins=50, alpha=0.7, color='orange')
ax[1, 1].set_xlabel('Torque [N·m]')
ax[1, 1].set_title('Load Torque')

# Torque vs i_q scatter
scatter = ax[1, 2].scatter(df['i_q'], df['torque_e'], c=df['omega_e'], s=5, alpha=0.5, cmap='viridis')
ax[1, 2].set_xlabel('i_q [A]')
ax[1, 2].set_ylabel('Torque [N·m]')
ax[1, 2].set_title('Torque vs i_q (colored by speed)')
plt.colorbar(scatter, ax=ax[1, 2], label='Speed [rad/s]')

plt.suptitle('Signal Distributions', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Noise distribution

For testing the robustess of the angl observer there are multiple current measurements. The original data is the ideal simulation data. Then there are two current measurements with added quantization and noise. This way we can ensure that the algorithm works fine over different data qualities. The observer is working in alpha,beta frame, this is why here are the alpha beta currents and the distribution. The voltages are not observed, because normally there is no noise. Normal foc observer are taking currents from the adc and voltages diretly out of the algorithm, so no measurement noise, it is the reference of the controller.

In [ ]:
# Noise analysis
fig, ax = plt.subplots(2, 2, figsize=(15, 4))
ax = ax.flatten()

# Current noise 12-Bit
i_alpha_noise_12bit = df['i_alpha_meas_12'] - df['i_alpha']
i_beta_noise_12bit = df['i_beta_meas_12'] - df['i_beta']

ax[0].hist(i_alpha_noise_12bit, bins=50, alpha=0.7)
ax[0].axvline(x=0, color='black', linestyle='--', linewidth=1)
ax[0].set_xlabel('Noise [A]')
ax[0].set_ylabel('Count')
ax[0].set_title(f'i_alpha Noise (12-Bit ADC) (std: {i_alpha_noise_12bit.std():.4f} A)')

ax[1].hist(i_beta_noise_12bit, bins=50, alpha=0.7, color='red')
ax[1].axvline(x=0, color='black', linestyle='--', linewidth=1)
ax[1].set_xlabel('Noise [A]')
ax[1].set_title(f'i_beta Noise (12-Bit ADC) (std: {i_beta_noise_12bit.std():.4f} A)')

# Current noise 8-Bit
i_alpha_noise_8bit = df['i_alpha_meas_8'] - df['i_alpha']
i_beta_noise_8bit = df['i_beta_meas_8'] - df['i_beta']

ax[2].hist(i_alpha_noise_8bit, bins=50, alpha=0.7)
ax[2].axvline(x=0, color='black', linestyle='--', linewidth=1)
ax[2].set_xlabel('Noise [A]')
ax[2].set_ylabel('Count')
ax[2].set_title(f'i_alpha Noise (8-Bit ADC) (std: {i_alpha_noise_8bit.std():.4f} A)')

ax[3].hist(i_alpha_noise_8bit, bins=50, alpha=0.7, color='red')
ax[3].axvline(x=0, color='black', linestyle='--', linewidth=1)
ax[3].set_xlabel('Noise [A]')
ax[3].set_title(f'i_beta Noise (8-Bit ADC) (std: {i_alpha_noise_8bit.std():.4f} A)')



plt.suptitle('Current Sensor Noise Analysis', fontsize=14)
plt.tight_layout()
plt.show()

print("Noise Statistics:")
print(f"i_alpha noise STD (12-Bit ADC): {i_alpha_noise_12bit.std():.6f} A")
print(f"i_alpha noise STD (12-bit ADC): {i_beta_noise_12bit.std():.6f} A")
print(f"i_alpha noise STD (8-Bit ADC): {i_alpha_noise_8bit.std():.6f} A")
print(f"i_alpha noise STD (8-bit ADC): {i_beta_noise_8bit.std():.6f} A")


In [ ]:
# Noise analysis
fig, ax = plt.subplots(2, 2, figsize=(15, 4))
ax = ax.flatten()

# Current noise 12-Bit
sin_theta_noise_12_bit = df['sin_theta_e_12bit'] - df['sin_theta_e']
cos_theta_noise_12_bit = df['cos_theta_e_12bit'] - df['cos_theta_e']

ax[0].hist(sin_theta_noise_12_bit, bins=50, alpha=0.7)
ax[0].axvline(x=0, color='black', linestyle='--', linewidth=1)
ax[0].set_xlabel('Noise [A]')
ax[0].set_ylabel('Count')
ax[0].set_title(f'Encoder Noise (sin(theta): 12-Bit Encoder) (std: {sin_theta_noise_12_bit.std():.4f} rad)')

ax[1].hist(cos_theta_noise_12_bit, bins=50, alpha=0.7, color='red')
ax[1].axvline(x=0, color='black', linestyle='--', linewidth=1)
ax[1].set_xlabel('Noise [A]')
ax[1].set_title(f'Encoder Noise (cos(theta): 12-Bit Encoder) (std: {cos_theta_noise_12_bit.std():.4f} rad)')

# Current noise 8-Bit
sin_theta_noise_8_bit = df['sin_theta_e_8bit'] - df['sin_theta_e']
cos_theta_noise_8_bit = df['cos_theta_e_8bit'] - df['cos_theta_e']

ax[2].hist(sin_theta_noise_8_bit, bins=50, alpha=0.7)
ax[2].axvline(x=0, color='black', linestyle='--', linewidth=1)
ax[2].set_xlabel('Noise [A]')
ax[2].set_ylabel('Count')
ax[2].set_title(f'Encoder Noise (sin(theta): 8-Bit Encoder) (std: {sin_theta_noise_8_bit.std():.4f} rad)')

ax[3].hist(cos_theta_noise_8_bit, bins=50, alpha=0.7, color='red')
ax[3].axvline(x=0, color='black', linestyle='--', linewidth=1)
ax[3].set_xlabel('Noise [A]')
ax[3].set_title(f'Encoder Noise (cos(theta): 8-Bit Encoder) (std: {cos_theta_noise_8_bit.std():.4f} rad)')



plt.suptitle('Angle Sensor Noise Analysis', fontsize=14)
plt.tight_layout()
plt.show()

print("Noise Statistics:")
print("Encoder Noise (sin(theta): 12-Bit Encoder):")
print(f"  {sin_theta_noise_12_bit.std():.6f} rad,  {math.degrees(sin_theta_noise_12_bit.std()):.6f}°")
print("Encoder Noise (cos(theta): 12-Bit Encoder):")
print(f"  {cos_theta_noise_12_bit.std():.6f} rad,  {math.degrees(cos_theta_noise_12_bit.std()):.6f}°")
print("Encoder Noise (sin(theta): 8-Bit Encoder):")
print(f"  {sin_theta_noise_8_bit.std():.6f} rad,  {math.degrees(sin_theta_noise_8_bit.std()):.6f}°")
print("Encoder Noise (cos(theta): 8-Bit Encoder):")
print(f"  {cos_theta_noise_8_bit.std():.6f} rad,  {math.degrees(cos_theta_noise_8_bit.std()):.6f}°")

# Correlation analysis
Here is a small analysis if the project is even possible. From motor physics i know that its possible to recreate angle from alpha beta currents and voltages. Here is now a correlation analysis so we know that it is also possible from the simulated data.

In [ ]:
# Cell: Minimal correlation matrix
print("PMSM Electrical Variables Correlation")

corr_cols = ['sin_theta_e', 'cos_theta_e', 'i_alpha', 'i_beta', 'v_alpha', 'v_beta']
corr = df[corr_cols].corr()

fig, ax = plt.subplots(figsize=(9, 7))

# Simple heatmap
im = ax.imshow(corr, cmap='RdBu', vmin=-1, vmax=1)

# Add values
for i in range(len(corr_cols)):
    for j in range(len(corr_cols)):
        ax.text(j, i, f'{corr.iloc[i, j]:.2f}',
                ha='center', va='center', fontsize=9)

# Labels
ax.set_xticks(range(len(corr_cols)))
ax.set_yticks(range(len(corr_cols)))
ax.set_xticklabels(corr_cols, rotation=45, ha='right')
ax.set_yticklabels(corr_cols)

plt.colorbar(im, ax=ax, shrink=0.8)
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

print("\n Quick Interpretation:")
print("\nNegative = opposite movement")
print("Positive = same direction movement")

## Summary of EDA

In [ ]:

print("\n Dataset Info:")
print(f"Total samples: {len(df):,}")
print(f"Time duration: {df['time'].max():.2f} seconds")
print(f"Sampling rate: {1/(df['time'].iloc[1] - df['time'].iloc[0]):.0f} Hz")

print("\n Target Variables (sin_theta, cos_theta):")
print(f"  Range: [{df['sin_theta_e'].min():.4f}, {df['sin_theta_e'].max():.4f}]")
print(f"  Mean: {df['sin_theta_e'].mean():.4f}")
print(f"  Std: {df['sin_theta_e'].std():.4f}")

print("\n Feature Ranges:")
print(f"  i_alpha: [{df['i_alpha'].min():6.2f}, {df['i_alpha'].max():6.2f}] A")
print(f"  i_beta: [{df['i_beta'].min():6.2}, {df['i_beta'].max():6.2f}] A")
print(f"  v_alpha: [{df['v_alpha'].min():6.2f}, {df['v_alpha'].max():6.2f}] V")
print(f"  v_beta: [{df['v_beta'].min():6.2}, {df['v_beta'].max():6.2f}] V")